In [1]:
#Importation of all the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn as sns
import math
import json
import zipfile  
import ssl
from scipy import stats

#To dowload data
import requests
import io
import gzip

#To create the mapchart
import iso3166
import plotly
from iso3166 import countries
import plotly.graph_objects as go

/Users/arthurnussbaumer/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data_folder = 'data_2/'

#Download mobility and intervention files
google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')
interventions = pd.read_csv(data_folder+'interventions.csv.zip')

#Download file with covid pageviews #MODIFY WITH THE NEW DATA
file = gzip.open(data_folder+'aggregated_timeseries.json.gz')
data = json.load(file)

<ipython-input-2-1fabc272e678>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')


In [ ]:
#Dataframe for each language and number of covid pages per day
covid_pages_desktop = {}
covid_pages_mobile = {}
covid_pages = {}
for language in interventions['lang']:#MODIFY WITH ALL THE LANGUAGES WE HAVE
    #Extract number of covid pages search on desktop per day per language from 2020-01-22 and 2020-07-31
    covid_pages_desktop[language] = pd.DataFrame.from_dict(data[str(language)]["covid"]["sum"], orient='index')
    covid_pages_desktop[language] = (covid_pages_desktop[language]).loc[(covid_pages_desktop[language].index < '2020-08-01') & (covid_pages_desktop[language].index >= "2020-01-22")] #to have the same window as for covid deaths/cases
    
    #Transform index of each data frame to datetime format
    covid_pages_desktop[language].index = pd.to_datetime(covid_pages_desktop[language].index)
    
    #Extract number of covid pages search on mobile devices per day per language from 2020-01-22 and 2020-07-31
    covid_pages_mobile[language] = pd.DataFrame.from_dict(data[str(language)+".m"]["covid"]["sum"], orient='index')
    covid_pages_mobile[language] = (covid_pages_mobile[language]).loc[(covid_pages_mobile[language].index < '2020-08-01') & (covid_pages_mobile[language].index >= "2020-01-22")]#to have the same window as for covid deaths/cases

    #Transform index of each data frame to datetime format
    covid_pages_mobile[language].index = pd.to_datetime(covid_pages_mobile[language].index)
    
    
    #Create dateframe for desktop and mobile searches
    covid_pages[language] = covid_pages_mobile[language] + covid_pages_desktop[language]
    
    #Transform index of each data frame to datetime format
    covid_pages[language].index = pd.to_datetime(covid_pages[language].index)

In [3]:
#Change name of the columns
names = ["Retail and Recreations", "Grocery and Pharmacy", "Parks","Transit stations", "Workplace", "Residential"]
google_mobility = google_mobility.rename(dict(zip(google_mobility.columns[8:],names)), axis = 1)

#Transform date string to datetime
google_mobility['date'] = pd.to_datetime(google_mobility['date'])

#Delete columns about region
google_mobility = google_mobility.drop(google_mobility.iloc[:,2:7], axis = 1)

#Keep same dates as for corona wikipedia pages
google_mobility = google_mobility.loc[(google_mobility['date'] < "2020-08-01") & (google_mobility['date'] > "2020-01-21")]

In [4]:
#Regroup all data per country and per date and take the average
mobility = google_mobility.groupby(["country_region_code","country_region","date"]).mean()

In [6]:
mobility

Retail and Recreations  \
country_region_code country_region       date                                 
AE                  United Arab Emirates 2020-02-15                0.625000   
                                         2020-02-16               -0.250000   
                                         2020-02-17               -2.000000   
                                         2020-02-18               -2.125000   
                                         2020-02-19               -2.250000   
...                                                                     ...   
ZW                  Zimbabwe             2020-07-27              -27.800000   
                                         2020-07-28              -24.800000   
                                         2020-07-29              -22.000000   
                                         2020-07-30              -30.200000   
                                         2020-07-31              -72.166667   

                                                     Grocery and Pharmacy  \
country_region_code country_region       date                               
AE                  United Arab Emirates 2020-02-15                 4.125   
                                         2020-02-16                 3.000   
                                         2020-02-17                 0.500   
                                         2020-02-18                 1.000   
                                         2020-02-19                -0.625   
...                                                                   ...   
ZW                  Zimbabwe             2020-07-27               -16.200   
                                         2020-07-28               -15.000   
                                         2020-07-29                -4.400   
                                         2020-07-30                -5.200   
                                         2020-07-31               -56.600   

                                                         Parks  \
country_region_code country_region       date                    
AE                  United Arab Emirates 2020-02-15   5.142857   
                                         2020-02-16   2.571429   
                                         2020-02-17   2.285714   
                                         2020-02-18   3.142857   
                                         2020-02-19   3.857143   
...                                                        ...   
ZW                  Zimbabwe             2020-07-27 -32.400000   
                                         2020-07-28 -31.000000   
                                         2020-07-29 -18.666667   
                                         2020-07-30 -33.800000   
                                         2020-07-31 -44.750000   

                                                     Transit stations  \
country_region_code country_region       date                           
AE                  United Arab Emirates 2020-02-15          2.833333   
                                         2020-02-16          3.166667   
                                         2020-02-17          3.000000   
                                         2020-02-18          1.833333   
                                         2020-02-19         -0.500000   
...                                                               ...   
ZW                  Zimbabwe             2020-07-27        -45.600000   
                                         2020-07-28        -40.600000   
                                         2020-07-29        -39.000000   
                                         2020-07-30        -54.400000   
                                         2020-07-31        -80.714286   

                                                     Workplace  Residential  
country_region_code country_region       date                                
AE                  United Arab Emirates 2020-02-15   2.87

## DEFINE LANGUAGE COUNTRY WHICH IS A DICTIONARY RELATING THE COUNTRY TO THE LANGUAGE + OUR_COUNTRIES

In [5]:
#Creating plot of the evolution per country of the mobility from baseline
for country in our_countries:
    correlation_table = pd.DataFrame({"Pearson correlation for {}".format(str(country)):["Correlation coefficient:","P-value:"]})
    for name in names:
        corr, p_value = stats.pearsonr(mobility[mobility.index.get_level_values('country_region')==country][name], covid_pages_mobile[language_country[country]][0] + covid_pages_desktop[language_country[country]][0])
        output_df = pd.DataFrame({str(name):[corr, p_value] })
        correlation_table = pd.concat([correlation_table, output_df], axis=1)
    print(correlation_table)

NameError: name 'our_countries' is not defined